In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 2, 20)

In [2]:
sql = '''
SELECT S.* , B.net AS cost, B.qty, T.name
FROM sells S
JOIN buys B
ON buy_id = B.id
JOIN stocks T
ON B.stock_id = T.id
WHERE S.date = '%s'
ORDER BY T.name
'''
sql = sql % today
sells = pd.read_sql(sql, conpf)
sells

,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id,cost,qty,name
0,889,284,2021-12-27,14.70,304.29,21.30,146674.41,1949,-20695.48,-12.36512,-2.31568,111,None,0,167369.89,10000,MCS
1,890,1085,2021-12-27,58.75,72.97,5.11,35171.92,285,-4966.78,-12.37404,-15.84745,110,None,0,40138.70,600,PTTGC


In [3]:
grouped = sells.groupby('name').agg(
{'qty':'sum',
 'price':'mean',
 'net':'sum',
 'cost':'sum',
 'profit':'sum'})
grouped

,qty,price,net,cost,profit
name,,,,,
MCS,10000,14.70,146674.41,167369.89,-20695.48
PTTGC,600,58.75,35171.92,40138.70,-4966.78


In [4]:
grouped.net.sum(),grouped.cost.sum(),grouped.profit.sum()

(181846.33000000002, 207508.59000000003, -25662.26)

In [5]:
grouped.index

Index(['MCS', 'PTTGC'], dtype='object', name='name')

In [ ]:
### Update table buy in stock database

In [6]:
names = sells.name.unique()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'MCS', 'PTTGC'"

In [4]:
in_p = "'KCE'"
in_p

"'KCE'"

In [5]:
sql = """
SELECT * 
FROM buy 
WHERE name IN (%s) 
ORDER BY name
"""
sql = sql % in_p
df = pd.read_sql(sql, const)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,KCE,2021-08-11,10000.0,77.25,0.0,0.0,1,1.0,1,A4


In [9]:
new_volume = 5400
sqlUpd = '''
UPDATE buy
SET volbuy = %s
WHERE name IN (%s)'''
sqlUpd = sqlUpd % (new_volume, in_p)
print(sqlUpd)


UPDATE buy
SET volbuy = 5400
WHERE name IN ('PTTGC')


In [6]:
sqlUpd = '''
UPDATE buy
SET date = '2021-10-07'
WHERE name IN (%s)'''
sqlUpd = sqlUpd % (in_p)
print(sqlUpd)


UPDATE buy
SET date = '2021-10-07'
WHERE name IN ('KCE')


In [7]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### Update table dividend in stock database

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name IN (%s) 
ORDER BY name
"""
sql = sql % in_p
df = pd.read_sql(sql, const)
df

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,PTTGC,1.0,0.0,2.0,0.0,3.0,0.0,0.0,5400.0,2021-09-03,2021-09-22,None,0


In [13]:
sqlUpd = '''
UPDATE dividend
SET shares = %s
WHERE name IN (%s)'''
sqlUpd = sqlUpd % (new_volume, in_p)
print(sqlUpd)


UPDATE dividend
SET shares = 5400
WHERE name IN ('PTTGC')


In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete sales and update stocks in PortLite

In [19]:
sql = """
SELECT * 
FROM stocks
WHERE name IN (%s) 
ORDER BY name
"""
sql = sql % in_p
df_stocks = pd.read_sql(sql, conlite)
df_stocks

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,135,PTTGC,70,54.25,S,0,58.75,1279.94,1.17,66.75,1000,-11,9,5400,3,12,CUT,SET50


In [20]:
sqlUpd = """
UPDATE stocks
SET available_qty = %s
WHERE name IN (%s)"""
sqlUpd = sqlUpd % (new_volume, in_p)
print(sqlUpd)


UPDATE stocks
SET available_qty = 5400
WHERE name IN ('PTTGC')


In [18]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

### Trace bakc process

In [21]:
#grouped.reset_index(level=0, inplace=True)
grouped.reset_index(level='name', inplace=True)
#level=['name', 'etc']
grouped

,name,qty,price,net,cost,profit
0,MCS,10000,14.70,146674.41,167369.89,-20695.48
1,PTTGC,600,58.75,35171.92,40138.70,-4966.78


In [22]:
sql = '''
SELECT *
FROM price
WHERE date = '%s'
AND name IN (%s)
'''
sql = sql % (today, in_p)
print(sql)


SELECT *
FROM price
WHERE date = '2021-12-27'
AND name IN ('PTTGC')



In [23]:
prices = pd.read_sql(sql, const)
prices

,name,date,price,maxp,minp,qty,opnp
0,PTTGC,2021-12-27,57.75,58.75,57.75,5498189,58.5


In [24]:
df_merge = pd.merge(grouped, prices, on='name', how='inner')
df_merge.dtypes

name        object
qty_x        int64
price_x    float64
net        float64
cost       float64
profit     float64
date        object
price_y    float64
maxp       float64
minp       float64
qty_y        int64
opnp       float64
dtype: object

In [25]:
cols = 'name qty_x price_x price_y'.split()
df_merge[cols]

,name,qty_x,price_x,price_y
0,PTTGC,600,58.75,57.75


In [26]:
df_merge['diff'] = df_merge.price_x - df_merge.price_y
df_merge['profit'] = (df_merge.price_x - df_merge.price_y) * df_merge.qty_x
cols = 'name qty_x price_x price_y diff profit'.split()
df_merge[cols]

,name,qty_x,price_x,price_y,diff,profit
0,PTTGC,600,58.75,57.75,1.0,600.0


In [27]:
df_merge.profit.sum()

600.0

In [28]:
df_merge['diff2'] = df_merge.opnp - df_merge.price_y
df_merge['profit2'] = (df_merge.opnp - df_merge.price_y) * df_merge.qty_x
cols = 'name qty_x opnp price_y diff2 profit2'.split()
df_merge[cols]

,name,qty_x,opnp,price_y,diff2,profit2
0,PTTGC,600,58.5,57.75,0.75,450.0


In [29]:
df_merge.profit2.sum()

450.0

In [19]:
pd.read_sql_query ('select T.name, B.* from buys B JOIN stocks T ON stock_id = T.id ORDER BY B.id DESC LIMIT 1', conpf)

,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,IMH,1087,199,2021-12-08,3000,11.0,68.31,4.78,Sold,33073.09,DTD,None


In [27]:
pd.read_sql_query ('select T.name, S.*, B.* from sells S JOIN buys B ON S.buy_id = B.ID JOIN stocks T ON stock_id = T.id ORDER BY S.id DESC LIMIT 1', conpf)

,name,id,buy_id,date,price,fee,vat,net,days,profit,...,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,VNT,894,986,2021-12-30,38.5,358.63,25.1,172866.27,139,3742.51,...,142,2021-08-13,4500,37.5,349.31,24.45,Sold,169123.76,DTD,None


In [14]:
tmp = pd.read_sql_query ('select stock_id from buys ORDER BY id DESC LIMIT 1', conpf)
stock_id_sr = tmp.stock_id
stock_id = stock_id_sr.to_string(index=False)
stock_id

'199'

In [15]:
pd.read_sql_query ('select * from stocks WHERE id = 199', conpf)

,id,name,category_id,website
0,199,IMH,4,


In [6]:
pd.read_sql_query ('select stock_id from buys ORDER BY id DESC LIMIT 1', conpf)

,stock_id
0,199
